# IS20140 Final Project - Overview

## The Idea:

To create a version of Mat2Vec trained on data from the social sciences as opposed to data from the material sciences.

## The major issue:

Transformer models have poor Theory of Mind, thus they struggle in the social sciences (citation needed)

## The Alternative:

Use fine-tuned models to gain a broad understanding of large subsets of articles, and make it easier to refine a selection.

## The Implementation: 

NB Need to grab the citation for T-5

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install all required files

In [ ]:
#Import modules required to run 8-bit T-5 network
!pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet accelerate
!pip install --quiet sentencepiece
!pip install --quiet tokenizers
!pip install --quiet datasets
!pip install --quiet evaluate
!pip install --quiet torch
!pip install --quiet numpy
!pip install pandas==1.3.4 #Force version 1.3.4 as read_excel fails otherwise
!pip install --quiet sentence-transformers
!pip install --quiet sklearn
!pip uninstall xlrd -y
!pip install xlrd==1.2.0

^C
ERROR: Operation cancelled by user


## Preprocess actual data

In [ ]:
#Import and preprocess dataset
import pandas as pd
#dataset = pd.read_excel("/content/drive/MyDrive/College/IS20140/Final Project (1)/savedrecs-tagSociology_HighlyCited-date26092022-1.xlsx")
dataset = pd.read_excel("/content/drive/MyDrive/College/GEOG30370/savedrecs-for_Geography_project.xls")
dataset = dataset.dropna(axis=0, subset=["Abstract"]) #Drop rows with no known abstract
#print(dataset.head())
print(dataset)

    Publication Type                                            Authors  \
0                  J  van Puijenbroek, PJTM; Buijse, AD; Kraak, MHS;...   
1                  J                Rossetti, G; Viaroli, P; Ferrari, I   
2                  J  Fernandes, MR; Aguiar, FC; Martins, MJ; Rivaes...   
3                  J  Sheng, Q; Xu, W; Chen, L; Wang, L; Wang, YD; L...   
4                  C  Fan, KW; Fok, L; Ma, XB; Yeh, P; Cheng, DS; Le...   
..               ...                                                ...   
907                J                 Robb, DM; Pieters, R; Lawrence, GA   
908                J  Fu, CH; Xu, Y; Bundy, A; Gruss, A; Coll, M; He...   
909                J  Matson, L; Ng, GHC; Dockry, M; Nyblade, M; Kin...   
910                J  Schmeller, D; Bohm, M; Arvanitidis, C; Barber-...   
912                J  Kane, A; Monadjem, A; Aschenborn, HKO; Bildste...   

    Book Authors Book Editors Book Group Authors  \
0            NaN          NaN                Na

References: 

    @inproceedings{specter2020cohan,
      title={{SPECTER: Document-level Representation Learning using Citation-informed Transformers}},
      author={Arman Cohan and Sergey Feldman and Iz Beltagy and Doug Downey and Daniel S. Weld},
    booktitle={ACL},
    year={2020}
    }

### Use models as they are:

In [ ]:
#Guess topics using T5
# As it stands, we MIGHT be able to run a fixed question against each abstract and use that to generate a summary?
# It all feels a bit on the tenuous side. Heck, it all feels VERY on the tenuous side.
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Use a variant of T5 that has already been fine-tuned on question-answering tasks

tokenizer = AutoTokenizer.from_pretrained("MaRiOrOsSi/t5-base-finetuned-question-answering")
model = AutoModelForSeq2SeqLM.from_pretrained("MaRiOrOsSi/t5-base-finetuned-question-answering")

def get_topics(input, model=model, tokenizer=tokenizer, max_output_length=50):
    question = "What are the main themes in this article?"
    context = input
    context = str(context)
    fully_formed_input = "question: "+question+" context: "+context
    fully_formed_input = str(fully_formed_input)
    input_ids = tokenizer(fully_formed_input, return_tensors="pt").input_ids
    input_ids = input_ids.to('cpu')
    outputs = model.generate(input_ids, max_new_tokens=max_output_length)
    output_string = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_string

all_abstracts = dataset["Abstract"].to_numpy()
topics_list = list()
model = model
tokenizer = tokenizer
max_output_length = 50

for abstract in all_abstracts:
    topics = get_topics(abstract, model, tokenizer, max_output_length)
    topics_list.append(topics)

dataset["Topics"] = topics_list
print(dataset)

Downloading:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors


    Publication Type                                            Authors  \
0                  J  van Puijenbroek, PJTM; Buijse, AD; Kraak, MHS;...   
1                  J                Rossetti, G; Viaroli, P; Ferrari, I   
2                  J  Fernandes, MR; Aguiar, FC; Martins, MJ; Rivaes...   
3                  J  Sheng, Q; Xu, W; Chen, L; Wang, L; Wang, YD; L...   
4                  C  Fan, KW; Fok, L; Ma, XB; Yeh, P; Cheng, DS; Le...   
..               ...                                                ...   
907                J                 Robb, DM; Pieters, R; Lawrence, GA   
908                J  Fu, CH; Xu, Y; Bundy, A; Gruss, A; Coll, M; He...   
909                J  Matson, L; Ng, GHC; Dockry, M; Nyblade, M; Kin...   
910                J  Schmeller, D; Bohm, M; Arvanitidis, C; Barber-...   
912                J  Kane, A; Monadjem, A; Aschenborn, HKO; Bildste...   

    Book Authors Book Editors Book Group Authors  \
0            NaN          NaN                Na

In [ ]:
dataset

,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,...,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,UT (Unique WOS ID),Web of Science Record,Topics
0,J,"van Puijenbroek, PJTM; Buijse, AD; Kraak, MHS;...",NaN,NaN,NaN,"van Puijenbroek, Peter J. T. M.; Buijse, Antho...",NaN,NaN,Species and river specific effects of river fr...,RIVER RESEARCH AND APPLICATIONS,...,Environmental Sciences & Ecology; Water Resources,HH6OJ,NaN,"hybrid, Green Published",NaN,NaN,2022-11-02,WOS:000455850600007,0,Increasing diversity and diversification
1,J,"Rossetti, G; Viaroli, P; Ferrari, I",NaN,NaN,NaN,"Rossetti, Giampaolo; Viaroli, Pierluigi; Ferra...",NaN,NaN,ROLE OF ABIOTIC AND BIOTIC FACTORS IN STRUCTUR...,RIVER RESEARCH AND APPLICATIONS,...,Environmental Sciences & Ecology; Water Resources,497HJ,NaN,NaN,NaN,NaN,2022-11-02,WOS:000270049100002,0,zooplankton diversity and abundance
2,J,"Fernandes, MR; Aguiar, FC; Martins, MJ; Rivaes...",NaN,NaN,NaN,"Fernandes, Maria Rosario; Aguiar, Francisca C....",NaN,NaN,Long-term human-generated alterations of Tagus...,CATENA,...,Geology; Agriculture; Water Resources,KS7LP,NaN,NaN,NaN,NaN,2022-11-02,WOS:000518488500042,0,Human intervention and natural disturbances in...
3,J,"Sheng, Q; Xu, W; Chen, L; Wang, L; Wang, YD; L...",NaN,NaN,NaN,"Sheng, Qiang; Xu, Wang; Chen, Long; Wang, Lei;...",NaN,NaN,Effect of Urban River Morphology on the Struct...,SUSTAINABILITY,...,Science & Technology - Other Topics; Environme...,4B0HI,NaN,gold,NaN,NaN,2022-11-02,WOS:000845469900001,0,Conservation of natural rivers and floods.
4,C,"Fan, KW; Fok, L; Ma, XB; Yeh, P; Cheng, DS; Le...",NaN,"Shang, HQ",NaN,"Fan, K. W.; Fok, L.; Ma, X. B.; Yeh, P.; Cheng...",NaN,NaN,Field Investigation on Biodiversity in the Eas...,PROCEEDINGS OF THE 2ND INTERNATIONAL YELLOW RI...,...,Environmental Sciences & Ecology; Water Resources,BIL99,NaN,NaN,NaN,NaN,2022-11-02,WOS:000260656600012,0,"Water, geological, and environmental factors"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,J,"Robb, DM; Pieters, R; Lawrence, GA",NaN,NaN,NaN,"Robb, Daniel M.; Pieters, Roger; Lawrence, Gre...",NaN,NaN,Fate of turbid glacial inflows in a hydroelect...,ENVIRONMENTAL FLUID MECHANICS,...,Environmental Sciences & Ecology; Mechanics; M...,XN0XH,34966250.0,"Green Published, hybrid",NaN,NaN,2022-11-02,WOS:000699006000001,0,"Temperature, conductivity, and turbidity"
908,J,"Fu, CH; Xu, Y; Bundy, A; Gruss, A; Coll, M; He...",NaN,NaN,NaN,"Fu, Caihong; Xu, Yi; Bundy, Alida; Gruss, Arna...",NaN,NaN,Making ecological indicators management ready:...,ECOLOGICAL INDICATORS,...,Biodiversity & Conservation; Environmental Sci...,JE3DM,NaN,"Green Published, Green Accepted, hybrid, Green...",NaN,NaN,2022-11-02,WOS:000490574200003,0,Conservation and sustainability
909,J,"Matson, L; Ng, GHC; Dockry, M; Nyblade, M; Kin...",NaN,NaN,NaN,"Matson, Laura; Ng, G-H Crystal; Dockry, Michae...",NaN,NaN,Transforming research and relationships throug...,ENVIRONMENTAL SCIENCE & POLICY,...,Environmental Sciences & Ecology,PR5US,NaN,hybrid,NaN,NaN,2022-11-02,WOS:000607302000013,0,"Respect, respect, and value in life"
910,J,"Schmeller, D; Bohm, M; Arvanitidis, C; Barber-...",NaN,NaN,NaN,"Schmeller, Dirk S.; Bohm, Monika; Arvanitidis,...",NaN,NaN,Building capacity in biodiversity monitoring a...,BIODIVERSITY AND CONSERVATION,...,Biodiversity & Conservation; Environmental Sci...,FJ0YJ,NaN,Green Submitted,NaN,NaN,2022-11-02,WOS:000412437200001,0,"Research, policy and practice to tackle biodiv..."


In [ ]:
# Summarize the abstract using T5
# The main themes/topics bit is kinda already doing that. And to be honest that's all we really need.

# As it stands, we MIGHT be able to run a fixed question against each abstract and use that to generate a summary?
# It all feels a bit on the tenuous side. Heck, it all feels VERY on the tenuous side.
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Use a variant of T5 that has already been fine-tuned on question-answering tasks

tokenizer = AutoTokenizer.from_pretrained("MaRiOrOsSi/t5-base-finetuned-question-answering")
model = AutoModelForSeq2SeqLM.from_pretrained("MaRiOrOsSi/t5-base-finetuned-question-answering")

def get_topics(input, model=model, tokenizer=tokenizer, max_output_length=50):
    question = "What are the main themes in this article?"
    context = input
    fully_formed_input = "question: "+question+" context: "+context
    fully_formed_input = str(fully_formed_input)
    input_ids = tokenizer(fully_formed_input, return_tensors="pt").input_ids
    input_ids = input_ids.to('cpu')
    outputs = model.generate(input_ids, max_new_tokens=max_output_length)
    output_string = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_string

all_abstracts = dataset["Abstract"].to_numpy()
topics_list = list()
model = model
tokenizer = tokenizer
max_output_length = 50

for abstract in all_abstracts:
    topics = get_topics(abstract, model, tokenizer, max_output_length)
    topics_list.append(topics)

dataset["Topics"] = topics_list
print(dataset)

Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors


    Publication Type                                            Authors  \
0                  J  van Puijenbroek, PJTM; Buijse, AD; Kraak, MHS;...   
1                  J                Rossetti, G; Viaroli, P; Ferrari, I   
2                  J  Fernandes, MR; Aguiar, FC; Martins, MJ; Rivaes...   
3                  J  Sheng, Q; Xu, W; Chen, L; Wang, L; Wang, YD; L...   
4                  C  Fan, KW; Fok, L; Ma, XB; Yeh, P; Cheng, DS; Le...   
..               ...                                                ...   
907                J                 Robb, DM; Pieters, R; Lawrence, GA   
908                J  Fu, CH; Xu, Y; Bundy, A; Gruss, A; Coll, M; He...   
909                J  Matson, L; Ng, GHC; Dockry, M; Nyblade, M; Kin...   
910                J  Schmeller, D; Bohm, M; Arvanitidis, C; Barber-...   
912                J  Kane, A; Monadjem, A; Aschenborn, HKO; Bildste...   

    Book Authors Book Editors Book Group Authors  \
0            NaN          NaN                Na

In [ ]:
torch.cuda.empty_cache()

# **References (get these into the right citation style)**

https://huggingface.co/blog/hf-bitsandbytes-integration

https://discuss.huggingface.co/t/t5-finetuning-tips/684/5

https://colab.research.google.com/github/patil-suraj/exploring-T5/blob/master/T5_on_TPU.ipynb#scrollTo=QLGiFCDqvuil 